In [1]:
AOI = 'POLYGON ((-85.299088 40.339368, -85.332047 40.241477, -85.134979 40.229427, -85.157639 40.34146, -85.299088 40.339368))'
# AOI = "MULTIPOLYGON (((34.31089955761184 47.50618872034119, 25.17868046954648 47.416142188659364, 25.116778551806377 52.12068793690748, 34.04176538017641 52.33837054247748, 34.31089955761184 47.50618872034119))))"
START_DATE = "2020-05-01"
END_DATE = "2020-06-30"

REQUEST_ID = '0'

In [12]:
import os
import json
import time
import shutil
import rasterio
import rasterio.mask as riomask
import shapely
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pylab as plt

from utils import crop_raster, stitch_tiles

In [3]:
default_crs = 'EPSG:4326'
polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(
    gpd.GeoSeries([polygon]),
    columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")
aoi = gpd.read_file(aoi_filename)

In [4]:
tiles_folder = "work/lulc2021"

In [5]:
class_names = {
    1: "Water",
    2: "Trees",
    4: "Flooded vegetation",
    5: "Crops",
    7: "Built Area",
    8: "Bare ground",
    9: "Snow/Ice",
    10: "Clouds",
    11: "Rangeland"
}

In [6]:
files = []
for file in os.listdir(tiles_folder):
    try:
        raster_path = os.path.join(tiles_folder, file)
        raster_path = crop_raster(raster_path, aoi_filename)
        files.append(raster_path)
    except Exception as e:
        pass

In [7]:
raster_path = stitch_tiles(
    files, os.path.join(tiles_folder, "aoi_raster.tif"))

saved raster lulc2021/aoi_raster_merged.tif


In [8]:
for file in files:
    try:
        os.remove(file)
    except Exception as e:
        print(e)

In [9]:
with rasterio.open(raster_path) as src:
    img = src.read(1)
    mask = src.read_masks(1)
    profile = src.profile

In [10]:
NUM_CLASSES = len(class_names)
arr = np.array(range(0, NUM_CLASSES)) / NUM_CLASSES
colors = plt.cm.jet(arr)
colors[0] = (0,0,0,1)

labels = []

for label, name in enumerate(class_names):
#     print(label)
    class_area = len(img[img == label]) / 10 ** 4
    # convert list of float values into string representing color
    class_color = ",".join(list(map(lambda x: str(int(x)), colors[label][:-1] * 255)))
    if class_area != 0:
        labels.append({
            "color": class_color, 
            "name": class_names[name],
            "area": class_area
        })
    else:
        labels.append({
            "color": class_color, 
            "name": name
        })
labels = json.dumps(labels)
labels

'[{"color": "0,0,0", "name": "Water", "area": 4391.1926}, {"color": "0,0,254", "name": "Trees", "area": 374.6769}, {"color": "0,96,255", "name": "Flooded vegetation", "area": 543.0472}, {"color": "0,212,255", "name": 5}, {"color": "76,255,170", "name": "Built Area", "area": 6.6231}, {"color": "170,255,76", "name": "Bare ground", "area": 13799.4882}, {"color": "255,229,0", "name": 9}, {"color": "255,122,0", "name": "Clouds", "area": 763.3021}, {"color": "254,18,0", "name": "Rangeland", "area": 26.6523}]'

In [11]:
os.remove(aoi_filename)

In [13]:
NB_USER = os.getenv('NB_USER')
os.makedirs(f"/home/{NB_USER}/work/results/landcover", exist_ok=True)
tile_name = f"{REQUEST_ID}_{START_DATE}_{END_DATE}.tif"

In [15]:
shutil.move(
    raster_path, f"/home/{NB_USER}/work/results/landcover/{tile_name}")

'/home/jovyan/work/results/landcover/0_2020-05-01_2020-06-30.tif'